# Edit Distance, Assembly, and Overlaps

Dynamic Program can be used to find approximate occurences of a pattern sequence (p) in a reference text (t). Here, an algorithm was developed to calculate minimum edit distances between sample DNA sequences and a genome. In addition, methods were developed to efficiently find overlaps within a set of sequencing reads.

In [1]:
# First, writing a function to parse the genome file

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [3]:
# Genome parsing
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [7]:
# Writing function for calculating edit distances
def editDistance(p, t):
    ''' 
    Takes arguments p (pattern), t (text) and returns the edit distance of the best match between P and T
    '''
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    # Initialize first row and column of matrix
    for i in range(len(p)+1):
        D[i][0] = i
    for i in range(len(t)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if p[i-1] == t[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the minimum value in the bottom row of the matrix
    return min(D[-1])


Testing the algorithm on sample sequences below

In [10]:
p = 'GCGTATGC'
t = 'TATTGGCTATACGGTT'
editDistance(p,t)

2

In [11]:
p = 'GCTGATCGATCGTACG'
t = genome
editDistance(p,t)

3

In [12]:
p = 'GATTTACCAGATTGAG'
t = genome
editDistance(p,t)

2

In [13]:
# Writing a function to calculate overlaps

def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [130]:
# Writing a function to read the sequencing read fastq file

def readReads(filename):
    with open(filename, 'r') as f:
        reads = []
        read = ''
        for line in f:
            if line[0] == '@':
                if read != '':
                    reads.append(read)
                read = ''
                
            else:
                read += line.rstrip()
    reads = list(map(lambda x: x.split('+')[0], reads))
    return reads

In [154]:
# reading the fastq file
reads = readReads('ERR266411_1.for_asm.fastq')

In [81]:
from itertools import permutations

# Overlaps
The goal of the next section is to find overlaps of minimum size k-mer within the set of sequencing reads parsed above. Unfortunately, one can't simply call overlaps() on every pair of reads, because that would be extremeley slow. Instead, I'll create a set of k-mers and add in every read that contains each k-mer. This can then be used as an index to find overlaps.

In [150]:

def overlap_all_pairs(reads, k):
#     making a kmer_dict that contains reads that correspond to each kmer
    kmer_dict = {}
    for read in reads:
        for i in range(len(read)-k+1):
            kmer = read[i:i+k]
            if kmer in kmer_dict:
                kmer_dict[kmer].add(read)
            else:
                kmer_dict[kmer] = set([read])
#     finding overlaps for each read
    olaps = {}
    for read in reads:
        read_suffix = read[-k:]
        potential_overlaps = kmer_dict[read_suffix]
        for possible_pair in potential_overlaps:
            olen = overlap(read, possible_pair, min_length=k)
            if olen > 0 and read != possible_pair:
                olaps[(read,possible_pair)] = olen
    return olaps  

In [151]:
olaps = overlap_all_pairs(reads, 30)

In [152]:
len(olaps)

904640

In [153]:
len(set(olaps))

904640

In [149]:
len(set([pair[0] for pair in olaps]))

7160

The algorithms above successfully finds the total number of overlaps and unique overlaps in the set of reads using a set kmer size.